In [ ]:
import random

def select_elements(main_list, exclude_list, num_elements_to_select):
    # Make a copy of the main list to avoid modifying the original list
    remaining_elements = main_list.copy()
    selected_elements = []

    exclude_weight = 0.25

    # Initialize probabilities list
    probabilities = []

    # Iterate until the desired number of elements is selected or there are no more elements to select
    for _ in range(num_elements_to_select):
        # Calculate probabilities for remaining elements
        probabilities = []
        for element in remaining_elements:
            if element in exclude_list:
                # Assign lower probability to elements in the exclude list
                p = exclude_weight 
            else:
                # Assign higher probability to elements not in the exclude list
                p = 1 - exclude_weight
            probabilities.append(p)

        # Perform random selection based on probabilities without replacement
        selected_element = random.choices(remaining_elements, weights=probabilities, k=1)[0]
        selected_elements.append(selected_element)

        # Remove selected element from remaining elements
        remaining_elements.remove(selected_element)

    return selected_elements

# Example usage
main_list = [0,1,2]
exclude_list = [0]  # Elements to exclude from selection
num_elements_to_select = 1

selected_elements = select_elements(main_list, exclude_list, num_elements_to_select)
print("Selected elements:", selected_elements)

In [ ]:
for _ in range(10):
    selected_elements = select_elements(main_list, exclude_list, num_elements_to_select)
    print("Selected elements:", selected_elements)

### add to code

In [ ]:
from numba import njit
import numpy as np
from numba.typed import Dict
from numba import types
from numba import njit, types
from numba.typed import Dict, List
import random

@njit(fastmath=True)
def assign_values_to_pes_exclusion(num_nodes, node_updates_per_pe, number_pes, previous_nodes, previous_neighbours):
    
    all_nodes = np.arange(num_nodes)
    k = node_updates_per_pe * number_pes
    not_previous_nodes = [x for x in all_nodes if x not in previous_nodes] # all nodes not previous nodes
    selected_values = select_elements(not_previous_nodes, previous_neighbours, k)
    selected_values = np.array(selected_values, dtype=np.int64)

    np.random.shuffle(selected_values)
    
    # Distribute values among PEs
    PEs = {}
    for i in range(number_pes):
        start_index = i * node_updates_per_pe
        end_index = min((i + 1) * node_updates_per_pe, num_nodes)
        pe_values = selected_values[start_index:end_index]
        PEs[i] = pe_values

    return PEs

@njit(fastmath=True)
def select_elements(main_list, exclude_list, num_elements_to_select):
    # Make a copy of the main list to avoid modifying the original list
    remaining_elements = main_list.copy()
    selected_elements = []

    exclude_weight = 0.25

    # Initialize probabilities list
    probabilities = []

    # Iterate until the desired number of elements is selected or there are no more elements to select
    for _ in range(num_elements_to_select):
        # Calculate probabilities for remaining elements
        probabilities = []
        for element in remaining_elements:
            if element in exclude_list:
                # Assign lower probability to elements in the exclude list
                p = exclude_weight 
            else:
                # Assign higher probability to elements not in the exclude list
                p = 1 - exclude_weight
            probabilities.append(p)

        # Perform random selection based on probabilities without replacement
        selected_element = random.choices(remaining_elements, weights=probabilities, k=1)[0]
        selected_elements.append(selected_element)

        # Remove selected element from remaining elements
        remaining_elements.remove(selected_element)

    return selected_elements

num_nodes = 5
node_updates_per_pe = 1
number_pes = 1
previous_nodes = [0]
previous_nodes = np.array(previous_nodes, dtype=np.int64)
previous_neighbours = []
previous_neighbours = np.array(previous_neighbours, dtype=np.int64)
PEs = assign_values_to_pes_exclusion(num_nodes=num_nodes, 
                                     node_updates_per_pe=node_updates_per_pe, 
                                     number_pes=number_pes, 
                                     previous_nodes=previous_nodes,
                                     previous_neighbours=previous_neighbours)
print(PEs)

### editied

In [ ]:
from numba import njit
import numpy as np

@njit(fastmath=True)
def is_in(arr, values):
    result = np.zeros(arr.shape[0], dtype=np.bool_)
    for value in values:
        result |= (arr == value)
    return result

@njit(fastmath=True)
def weighted_random_choice(values, weights):
    cumulative_weights = np.cumsum(weights)
    total = cumulative_weights[-1]
    rand = np.random.random() * total
    for i in range(len(cumulative_weights)):
        if rand < cumulative_weights[i]:
            return values[i]

@njit(fastmath=True)
def select_elements(main_list, exclude_list, num_elements_to_select):
    remaining_elements = main_list.copy()  # Start directly with a NumPy array
    selected_elements = np.empty(num_elements_to_select, dtype=np.int64)  # Preallocate array
    
    for i in range(num_elements_to_select):
        # Manually check for exclusion using a Numba-compatible method
        in_exclude_list = is_in(remaining_elements, exclude_list)
        weights = np.where(in_exclude_list, 0.25, 0.75)
        selected_element = weighted_random_choice(remaining_elements, weights)
        selected_elements[i] = selected_element
        remaining_elements = remaining_elements[remaining_elements != selected_element]
    
    return selected_elements

@njit(fastmath=True)
def assign_values_to_pes_exclusion_neighbours_STOCHASTIC(num_nodes, node_updates_per_pe, number_pes, previous_nodes, previous_neighbours):
    all_nodes = np.arange(num_nodes)
    k = node_updates_per_pe * number_pes

    not_previous_nodes = np.array([x for x in all_nodes if x not in previous_nodes], dtype=np.int64)  # Ensure dtype here
    selected_values = select_elements(not_previous_nodes, previous_neighbours, min(len(not_previous_nodes), k))

    if selected_values.shape[0] < k:
        result = []
        for x in all_nodes:
            if x not in selected_values:
                result.append(x)  
        result = np.array(result, dtype=np.int64)
        random_values = np.random.choice(result, k - selected_values.shape[0], replace=False)
        for value in random_values:
            selected_values = np.append(selected_values, value)

    np.random.shuffle(selected_values)
    PEs = {}
    for i in range(number_pes):
        start_index = i * node_updates_per_pe
        end_index = min((i + 1) * node_updates_per_pe, num_nodes)
        pe_values = selected_values[start_index:end_index]
        PEs[i] = pe_values
    return PEs

# Function and variable setup
num_nodes = 10
node_updates_per_pe = 5
number_pes = 1
previous_nodes = np.array([0], dtype=np.int64)
previous_neighbours = np.array([1], dtype=np.int64)
PEs = assign_values_to_pes_exclusion_neighbours_STOCHASTIC(num_nodes=num_nodes, 
                                     node_updates_per_pe=node_updates_per_pe, 
                                     number_pes=number_pes, 
                                     previous_nodes=previous_nodes,
                                     previous_neighbours=previous_neighbours)
print(PEs)

In [ ]:
# Function and variable setup
num_nodes = 10
node_updates_per_pe = 1
number_pes = 5
previous_nodes = np.array([0], dtype=np.int64)
previous_neighbours = np.array([1], dtype=np.int64)
PEs = assign_values_to_pes_exclusion_neighbours_STOCHASTIC(num_nodes=num_nodes, 
                                     node_updates_per_pe=node_updates_per_pe, 
                                     number_pes=number_pes, 
                                     previous_nodes=previous_nodes,
                                     previous_neighbours=previous_neighbours)
print(PEs)

In [13]:
import random
import numpy as np

def generate_schedule():
    # Create lists of even and odd nodes
    even_nodes = [x for x in range(10) if x % 2 == 0]
    odd_nodes = [x for x in range(10) if x % 2 != 0]
    
    # Shuffle the even and odd lists
    random.shuffle(even_nodes)
    random.shuffle(odd_nodes)
    
    # Randomly decide whether to start with even or odd nodes
    if random.choice([True, False]):
        # Start with odd nodes first
        schedule = odd_nodes + even_nodes
    else:
        # Start with even nodes first
        schedule = even_nodes + odd_nodes
    
    return schedule

# Generate and print a random valid schedule
print(np.array(generate_schedule(), dtype=np.int64))

[4 2 6 0 8 3 9 7 1 5]
